## Part 1 – Post a hello message to Slack using the Web API ##

In [1]:
%pip install slack_sdk

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
%pip install dotenv

  Using cached dotenv-0.9.9-py2.py3-none-any.whl.metadata (279 bytes)
  Using cached python_dotenv-1.2.1-py3-none-any.whl.metadata (25 kB)
Using cached dotenv-0.9.9-py2.py3-none-any.whl (1.9 kB)
Using cached python_dotenv-1.2.1-py3-none-any.whl (21 kB)

   ---------------------------------------- 0/2 [python-dotenv]
   ---------------------------------------- 0/2 [python-dotenv]
   ---------------------------------------- 2/2 [dotenv]

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
from dotenv import load_dotenv
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

# 1. Dans un Notebook, on peut charger le .env directement s'il est à côté
# On utilise override=True pour être sûr de bien écraser les vieilles versions en mémoire
load_dotenv(override=True)

TOKEN_SLACK = os.environ.get("SLACK_BOT_TOKEN")
ID_CANAL = os.environ.get("CHANNEL_ID")

# --- PETIT TEST DE SÉCURITÉ ---
if TOKEN_SLACK is None:
    print("Erreur : Le Token n'est pas lu. Vérifie le fichier .env")
else:
    print(f"Token chargé : {TOKEN_SLACK[:10]}...")

# 2. Connexion
client = WebClient(token=TOKEN_SLACK)

try:
    print("Tentative d'envoi...")
    client.chat_postMessage(
        channel=ID_CANAL,
        text="Bonjour tout le monde ! je suis de nouveau actif)"
    )
    print("ÇA MARCHE !")
except SlackApiError as e:
    print(f"Erreur Slack : {e.response['error']}")

✅ Token chargé : xoxb-10255...
Tentative d'envoi...
🚀 ÇA MARCHE !


In [25]:
%pip install websocket-client

  Using cached websocket_client-1.9.0-py3-none-any.whl.metadata (8.3 kB)
Using cached websocket_client-1.9.0-py3-none-any.whl (82 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
def interaction():
    try:
        # 1. On récupère le dernier message
        resultat = client.conversations_history(channel=ID_CANAL, limit=1)
        messages = resultat.get("messages", [])
        
        if messages:
            msg = messages[0]
            texte = msg.get("text", "")
            user_id = msg.get("user", "quelqu'un")
            
            print(f"Dernier message lu : {texte}")

            # 2. Correction : on cherche 'BOT' en majuscules
            if "BOT" in texte.upper() and "bot_id" not in msg:
                print("Condition validée ! Envoi de la réponse...")
                client.chat_postMessage(
                    channel=ID_CANAL, 
                    text=f"Bonjour <@{user_id}> ! Je suis le bot du Groupe 11 et j'ai bien reçu ton message."
                )
                print("✅ Interaction réussie !")
            else:
                print("Condition non remplie (soit pas de mot 'bot', soit c'est déjà le bot qui parle).")
                
    except Exception as e:
        print(f"Erreur : {e}")

interaction()

Dernier message lu : hello bot
Condition validée ! Envoi de la réponse...
✅ Interaction réussie !


## Part 2 - boucle pour uploader les images dans le canal ##

In [16]:
# On définit le dossier (chemin relatif au Notebook)
dossier_source = "dossier_image" 

def envoyer_images():
    try:
        # On liste les fichiers présents
        fichiers = os.listdir(dossier_source)
        print(f"Analyse du dossier... {len(fichiers)} fichiers trouvés.")

        for nom in fichiers:
            chemin = os.path.join(dossier_source, nom)
            
            # On ne prend que les images
            if nom.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
                print(f"Envoi de {nom}...")
                client.files_upload_v2(
                    channel=ID_CANAL,
                    file=chemin,
                    initial_comment=f"Image automatique : {nom}"
                )
        print("✅ Toutes les images ont été envoyées !")
    except FileNotFoundError:
        print(f"❌ Erreur : Le dossier '{dossier_source}' n'existe pas.")
    except Exception as e:
        print(f"❌ Erreur lors de l'envoi : {e}")

# Exécution de la partie 2
envoyer_images()

Analyse du dossier... 3 fichiers trouvés.
Envoi de Lac-dAnnecy-avec-voilier.jpg...
Envoi de lacs-de-savoie-saint-bon-tarentaise-147184370784.jpg...
Envoi de MyForest_60f06173290292583608699b.jpeg...
✅ Toutes les images ont été envoyées !


## Part 3 - interaction avec le bot ##

In [11]:
%pip install wikipedia

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11785 sha256=8eee8f66b225a0cb1b4533ceb64d120aa4f98c00d00f913ca9624ddd8868327f
  Stored in directory: c:\users\matte\appdata\local\pip\cache\wheels\79\1d\c8\b64e19423cc5a2a339450ea5d145e7c8eb3d4aa2b150cde33b
Successfully built wikipedia
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import os
import wikipedia
from dotenv import load_dotenv
from slack_sdk import WebClient

# Configuration de la langue pour Wikipedia
wikipedia.set_lang("fr")

load_dotenv(override=True)
BOT_TOKEN = os.environ.get("SLACK_BOT_TOKEN")
ID_CANAL = os.environ.get("CHANNEL_ID")
client = WebClient(token=BOT_TOKEN)

# On garde en mémoire l'ID du dernier message traité pour ne pas répondre en boucle
last_processed_ts = None

def wiki_bot_interaction():
    global last_processed_ts
    try:
        # 1. On récupère le dernier message
        resultat = client.conversations_history(channel=ID_CANAL, limit=1)
        messages = resultat.get("messages", [])
        
        if messages:
            msg = messages[0]
            texte = msg.get("text", "")
            ts = msg.get("ts") # Timestamp unique du message
            
            # On vérifie si c'est un nouveau message et s'il contient "Wikipedia:"
            if ts != last_processed_ts and "Wikipedia:" in texte and "bot_id" not in msg:
                print(f"Requête détectée : {texte}")
                
                # Extraction du titre (on prend ce qu'il y a après le :)
                titre = texte.split("Wikipedia:")[1].strip()
                
                try:
                    # Recherche du résumé (1 seul paragraphe)
                    resume = wikipedia.summary(titre, sentences=1)
                    reponse = f"📚 *Wikipedia : {titre}*\n\n{resume}"
                except wikipedia.exceptions.DisambiguationError as e:
                    reponse = f"Trop de résultats pour '{titre}'. Sois plus précis !"
                except wikipedia.exceptions.PageError:
                    reponse = f"Désolé, je n'ai pas trouvé de page pour '{titre}'."
                except Exception:
                    reponse = "Une erreur est survenue lors de la recherche."

                # Envoi de la réponse sur Slack
                client.chat_postMessage(channel=ID_CANAL, text=reponse)
                last_processed_ts = ts # On marque ce message comme traité
                print("Réponse envoyée !")

    except Exception as e:
        print(f"Erreur : {e}")

# 3. Boucle de surveillance (le bot "attend" le prochain message)
print("⚡ WikiBot actif ! Écris 'Wikipedia:[titre]' dans Slack...")
try:
    while True:
        wiki_bot_interaction()
        import time
        time.sleep(5) # Il vérifie toutes les 5 secondes
except KeyboardInterrupt:
    print("Bot arrêté.")

⚡ WikiBot actif ! Écris 'Wikipedia:[titre]' dans Slack...
Requête détectée : Wikipedia:Révolution française
✅ Réponse envoyée !
Bot arrêté.
